In [1]:
import numpy as np
import scipy as sp
from fitness_funs_non_dim import *
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp


In [6]:
import numpy as np
import scipy as sp
from fitness_funs_non_dim import *

def group_formation_model_non_dim(T, F_of_x_vec,P,N1,N2, params):
    '''
    the full system of balance equations for x = 1,2,3,...,x_max
    non-dimensionalized
    @inputs:
    T - time, necessary for running with solve_ivp
    F_of_x_vec - a vector of the (scaled) number of groups of 
            size 1, 2, 3, ..., x_max (maximum group size)
    P - population size of predators
    N1 - population size of big prey
    N2 - population size of small prey
    params - is a dictionary of the parameters that must contain: 
            β1, β2, A1, H1, H2, η1, η2, α1_of_1, α2_of_1, s1, s2, limited_portions, 
            Tx, d, ξ, r, γ
    @ returns
    dF_dT for x = 1, 2, ..., xmax
    '''
    x_max = params['x_max']; Tx = params['Tx']; 
    η1 = params['η1']; η2 = params['η2']; tildeδ = 1 - η1 - η2
    d = params['d']
    ξ = params['ξ']
    
    F_of_x_vec = np.append(F_of_x_vec,0) # so can find dfdt at x = x_max
    
    def F(x):
        return F_of_x_vec[x-1]
    def S(x,y):
        return best_response_fun_given_fitness(x,y,fitnessvec,d)
    def ψ(x):
        F_of_1 = F_of_x_vec[0]
        if x== 1 and F_of_1 >=1:
            return ( ξ * F_of_1 - 1) * S(2,1)
        elif x <= x_max - 1:
            return ξ * F_of_1*S(x+1,1)
        else:
            return 0
    def ϕ(x):
        return x*S(1,x) if x <= x_max else 0
    
    xvec = np.arange(1,x_max+1,1)
    # it \tau_x > 0make population matrix = birth matrix + death matrix
    fitnessvec = fitness_from_prey_non_dim(xvec, N1, N2, **params)
    dFdT_vec = np.zeros(x_max)
    
    for x in xvec:
        if x == 1:
            dFdT = (2*F(2)*ϕ(2) + np.sum([F(y) * ϕ(y) for y in range(3,x_max+1)]) \
                    - sum([F(y-1)*ψ(y-1) for y in range(2,x_max+1)]))/Tx
        elif x == 2:
            dFdT = (-F(2)*ϕ(2) - F(2)*ψ(2) + 0.5*F(1)*ψ(1) + F(3)*ϕ(3))/Tx
        else:
            dFdT = (-F(x)*ϕ(x) - F(x) * ψ(x) + F(x-1)*ψ(x-1) + F(x+1)*ϕ(x+1))/Tx
        
        dFdT_vec[x-1] = dFdT
    return dFdT_vec
    

def fun_1_death(x, Tx, tildeδ):
    '''
    The probability of AT LEAST one death in a group of size x over time τ_x
    @inputs:
    x - grp size
    Tx - scaled group evolution time constant
    tildeδ - scaled death rate
    params - dictionary of parameters from the ret of the model, not really needed...
    @output:
    float

    @example
    >> fun_1-death(x=1, Tx = 0.01, tildeδ = 0.1)
    0.0010000000000000009
    >> funfdgf_1_death(np.array([1,2,3]), 0.01, 0.1)
    array([0.001   , 0.001999, 0.002997])sxz
    
    '''
    return 1 - (1 - δ*Tx)**x

def fun_death_y_to_x(x, y, Tx, tildeδ, x_max):
    '''
    The probability a group of size y shrinks to a group of size x because y - x individuals die, 
    works for for x < y, y <= x_max

    @inputs:
    x = group size after death, is the shape of what is being returned
    y = original group size, y > x, y <= x_max
    tildeδ = death rate
    Tx - time constant of group dynamics scaled to population dynamic time
    x_max = maximum group size
    params = dictionary of other parameters used in the model

    @output:
    float between 0 and 1 (note for τx small, fun_death_y_to_x(x,y,**params) \approx 0 if x < y-1

    @example
    >>fun_death_y_to_x(x=2, y=3, **dict(Tx=0.01, tildeδ=0.1, x_max=10))
    0.0029940030000000003
    '''
    if isinstance(y, np.ndarray):
        to_return = np.zeros(y.shape)
        notzero = x<y
        y = y[notzero]
        if isinstance(x, np.ndarray):
            x = x[notzero]
        to_return[notzero] = nchoosek(y,y-x) * \
                        (tildeδ*Tx)**(y-x)*(1-tildeδ*Tx)**x
        return to_return
    else:
        if x < y:
            return nchoosek(y,y-x) * (tildeδ*Tx)**(y-x)*(1-tildeδ*Tx)**x
            
def nchoosek(n,k):
    '''
    n choose k
    n!/(k!(n-k)!)
    @inputs:
    n and k are integers, but can handle np.arrays
    @returns:
    positive integer (or array if inputs are arrays
    @example
    >> nchoosek(3,1)
    3.0
    >> nchoosek(np.array([3,2]),1)
    array([3.,2.])
    '''
    return sp.special.factorial(n)/(sp.special.factorial(k) \
                                    * sp.special.factorial(n-k))

def fun_leave_group(x, fitnessvec, x_max, d):
    '''
    The probability an individual leaves a group of size x.
    This is ϕ(x) in the text
    @inputs
    x - current grp size (before leaving)
    fitnessvec = vector of fitnesses for each group size
    x_max - parameter, maximum group size
    d = parameter determining steepness of best response function

    @ example:
    >> fitnessvec = array([0.24166667, 0.45833333, 0.53055556])
    >> fun_leave_group(xvec=[1,2,3], fitnessvec, x_max=3, d=100)
    array([0.5       , 0.03915869, 0.01923075])
    '''
    # deciding between being alone and staying in group of size x
    return best_response_fun_given_fitness(1,x,fitnessvec,d)

def best_response_fun_given_fitness(x,y,fitnessvec, d):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    fitnessvec - vector of fitnesses fro x = 1, 2, ..., xmax
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model 
    @returns:
    float between 0 and 1
    
    '''
    W_of_x = fitnessvec[x-1]
    W_of_y = fitnessvec[y-1]
    return W_of_x**d/(W_of_x**d + W_of_y**d)
    
def best_response_fun(x,y, N1,N2, d, **params):
    '''
    Compares W(x) to W(y) to "decide" on group size y or x
    @inputs
    x - potentially new group size
    y - current grp size
    N1 - big prey pop size  normalized by carrying capacity
    N2 - small prey pop size normalized by carrying capacity
    d - steepness, or sensitivity, of best response function
    params - dictionary of params used by the rest of the model, 
            but must include all params relevant to functional responses and 
            inclusive fitness 
    @returns:
    float between 0 and 1

                            α1_of_1=0.05, α2_of_1=0.95, s1=2, s2=2) )
    4.4162891943422267e-07
    
    '''
    W_of_x = fitness_from_prey_non_dim(x, N1, N2, **params)
    W_of_y = fitness_from_prey_non_dim(y, N1, N2, **params)
    return W_of_x**d/(W_of_x**d + W_of_y**d)

In [3]:

def run_simulations_vary_param(P, N1, N2, param_vec, param_key, params, pop_process = False, 
                               t_f = 10, delta_t = 0.01,
                               initialstate = np.nan):
    '''
    runs the ode system for dfdt over a list of param values for one of the params (e.g. s_2, s_1, etc...)
    @inputs
    P, N1, N2 - scaled pop sizes of pred, big prey, small prey, respec
    param_vec = list of param values to run simulation for, corresponding to param_key
    param_key = name of parameter varied in param_vec
    params = dic of param values
    pop_process = True or False, whether or not to include death rate and birth rate
    t_f = final time for simulation (note that t_f = 10 is still many steps of group simulation
    delta_t = time step
    initialstate - optional input, list of length x_max of num grps of size 1, 2, 3, ..., x_max

    
    @returns: 
    final_f_of_x - array of f(x) vectors (for x = 1, 2, ..., x_max) 
                    for each parameter value (one row per param value)
    at_equilibrium - whether each f(x) vector is at the stationary distribution, 
        1 for at equilibrium, 0 for no
    
    '''
    x_max = params["x_max"]
    if not isinstance(initialstate,np.ndarray): # otherwise it's an array, and intiial state is given
        if np.isnan(initialstate):
            initialstate = np.array([P, *np.zeros(x_max - 1)])

    final_f_of_x = np.zeros((len(param_vec), x_max))
    at_equilibrium = np.zeros(len(param_vec))
    dfdt_final = np.zeros((len(param_vec), x_max))
    
    # run model for each parameter value
    for i, param in enumerate(param_vec):
        # put new parameter values into params dictionary
        if isinstance(param_key, list): # in case want to set multiple params to same value
            for pkey in param_key:
                params[pkey] = param
        else:
            params[param_key] = param

        # run the model using LSODA, like in odeint?
        tsteps = 100
        #t_eval =  np.arange(0.0, t_f, delta_t)
        #out = solve_ivp(group_formation_model_separate, [0, t_f], initialstate, method="RK45",
        #        args=(P, N1, N2, pop_process, params))
        out = solve_ivp(group_formation_model_non_dim, [0, t_f], initialstate, method="LSODA",
                args=(P, N1, N2, params))
        final_distribution = np.transpose(out.y)[-1]
        
        # check at equilibrium
        dfdt_final[i,:], at_equilibrium[i] = check_at_equilibrium(final_distribution, P, N1, N2,pop_process,**params)
        
        
        final_f_of_x[i,:] = final_distribution
        

    return final_f_of_x, at_equilibrium, dfdt_final



def run_simulations_vary_param2(P, N1, N2, param_vec1, param_vec2,
                                param_key1, param_key2, params, pop_process = False,
                                t_f = 1e5,
                               initialstate = np.nan):
    '''
    run_simulations_vary_param, but with 2 parameters you're varying!
    @inputs
    P, N1, N2 - pop sizes
    param_vec1 - list of values for first param to vary by
    param_vec2 - list of values for 2nd param 
    param_key1, param_key2 - names of 1st and 2nd params being varied, respec
    params - dic of other params in the model
    @returns: 
    list_of_f_of_x - list of f(x) arrays. The rows of the arrays corresponds to param1, the arrays themselves 
        correspond to param2
    list_at_equilibrium - says whether each f(x) vector is at the stationary distribution, 1 for at equilibrium, 
            0 for no
    list_of_df_dt - df_dt at final step of simulation
'''

    list_of_f_of_x = [] # should be a list of lists of lists
    list_at_equilibrium = []
    list_of_df_dt = []
    for i, param2 in enumerate(param_vec2):
        params[param_key2] = param2
        out = run_simulations_vary_param(P, N1, N2, param_vec1, 
                                         param_key1, params, pop_process = pop_process,  
                                         t_f = t_f, initialstate = initialstate)
        final_f_of_x, at_equilibrium, dfdt_final = out
        list_of_f_of_x.append(final_f_of_x) # note that final_f_of_x includes f(1)
        list_at_equilibrium.append(at_equilibrium)
        list_of_df_dt.append(dfdt_final)
    return list_of_f_of_x, list_at_equilibrium, list_of_df_dt
    
        
'''
    try:
        if x == 2:
            join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
            #join_smaller_grp = (1/2)*f(1)*(f(1)-1)*ψ(1)
        else:
            join_smaller_grp = f(1)*f(x-1)*ψ(x-1)
    except ValueError:
        join_smaller_grp = np.zeros(np.shape(x))
        join_smaller_grp = f(1)* f(x-1) * ψ(x-1)
        #join_smaller_grp[x==2] = (1/2)*f(1) *(f(1)-1)*ψ(1)
        #join_smaller_grp[x>2] = f(1)*f(x[x>2] - 1) * ψ(x[x>2] - 1)
'''
colors = ['k','r','b','cyan']
markers = ["o","","v", ""]

def format_ax(ax,xlab,ylab, xlim = None, ylim=None,
              fs_labs = 20, fs_legend = 20, if_legend = False):
    ax.set_xlabel(xlab, fontsize = fs_labs)
    ax.set_ylabel(ylab, fontsize = fs_labs)
    if xlim != None:
        ax.set_xlim(xlim)
    if ylim != None:
        ax.set_ylim(ylim)
    for s in ['top', 'right']:
        ax.spines[s].set_visible(False)
    if if_legend:
        ax.legend(fontsize = fs_legend)
''' 
changing a parameter value, but not x_max!!
len(param_vec) must be 3
example of linelabel: r'$\theta_2=$'+'%d'
param_key may be multiple keys, (the parameter version for big prey and small prey) 
    if they are being set to the same param value
initialstate (optional) is a vector of the number of groups of size 2, 3, ... , x_max
'''
def plot_f_of_x(P, N1, N2,param_vec, param_key, linelabel, params, pop_process = False, 
                delta_t = 0.01, t_f = 10, initialstate = np.nan):
    
    # set up figure
    
    fig,ax = plt.subplots(1,1,figsize = (6,5))

    # run simulation for each parameter
    out = run_simulations_vary_param(P, N1, N2, param_vec, param_key, params, delta_t = delta_t, t_f = t_f,
                               initialstate = np.nan)
    
    final_f_of_x, at_equilibrium, dfdt_final = out

    # plot the distribution f(x)
    x_max = params["x_max"]
    for i, param in enumerate(param_vec):
        full_f_vec = final_f_of_x[i]
        ax.plot(np.arange(1,x_max+1,1),full_f_vec,color = colors[i], marker = markers[i],label = linelabel %param)

    # format the figure
    format_ax(ax,xlab = r'Group Size $x$', ylab = r'Num. groups of size $x$, $f(x)$', 
              xlim=[0,x_max], if_legend = True)

    
    return fig, ax, final_f_of_x, at_equilibrium, dfdt_final


    
def plot_mean_grp_membership(param_vec, list_f_of_x, x_max, xlab, type = 'bar'):
    
    mean_group_size_membership_vec = [mean_group_size_membership(f_vec,p,x_max)\
                                      for f_vec in list_f_of_x]
    fig,ax = plt.subplots(1,1,figsize=(6,5))
    if type == 'bar':
        param_vec_str = [str(param) for param in param_vec]
        plt.bar(param_vec_str,mean_group_size_membership_vec, width = 0.3)
        for i,bar in enumerate(ax.patches):
            ax.text(bar.get_x()+0.05,bar.get_height()+0.2, 
                 str(round(mean_group_size_membership_vec[i],2)),
                fontsize = 10, fontweight = 'bold')
    elif type == 'line':
        plt.plot(param_vec, mean_group_size_membership_vec)
    ymax = np.max(mean_group_size_membership_vec)+0.5
    format_ax(ax,xlab=xlab,ylab='Mean Group Membership',ylim=[0,ymax])

    return fig, ax, mean_group_size_membership_vec
    
    
    

'''
plots the number of individuals in groups of size x (the x-axis is, y-axis is number of individuals)

'''
def plot_group_membership_hist(param_vec, f_vec_list, x_max, linelabel):
    
    fig,ax = plt.subplots(1,1,figsize = (6,5))
    for i,f_vec in enumerate(f_vec_list):
        xvec = list(range(1,x_max+1))
        membership_vec = [x*f_vec[x-1] for x in xvec]
        ax.plot(xvec, membership_vec, label = linelabel %param_vec[i], c = colors[i])
    format_ax(ax, xlab='Group Size', ylab = 'Number of Individuals', if_legend=True)

    return fig, ax



def plot_mean_grp_membership_2params(P, N1, N2, param_key1, param_vec1,
                                     param_key2, param_vec2, params, linelabel, xlabel,
                                     t_f = 1, f_of_x_vec = np.nan, ylim = None):
    '''
    Plot mean group size an individual is part of versus 2 paramers
    param1 is on the x-axis, param2 is the color of the lines
    can only have 3 values for param2
    @returns: fig, ax, list_at_equilibrium, list_of_meanx
    '''
    
    fig, ax = plt.subplots(1,1,figsize = (5,6))
    x_max = params['x_max']

    # simulate
    out = run_simulations_vary_param2(P, N1, N2, param_vec1, param_vec2,
                                param_key1, param_key2, params, 
                                      t_f = t_f, initialstate = f_of_x_vec)
    list_of_f_of_x, list_at_equilibrium, _ = out
    list_of_meanx = [[mean_group_size_membership(f_vec,p,x_max)\
                                      for f_vec in list_f] \
                 for list_f in list_of_f_of_x]
    # plot the simulation
    colors = ['k','r','b','cyan']
    for i, param2 in enumerate(param_vec2):
        plt.plot(param_vec1, list_of_meanx[i], c = colors[i], label = linelabel%param2)

    # format the plot
    format_ax(plt.gca(),xlab = xlabel,
          ylab = 'Expected Group Size\nof Individual', 
          xlim = None, ylim=ylim, fs_labs = 20, fs_legend = 16, if_legend = True)
    return fig, ax, list_at_equilibrium, list_of_meanx, list_of_f_of_x
    
'''
This plots different aspects of the model(specified by funhandle) that change with group size
'''
def plot_measure_vs_grp_size(paramkey, param_vals, linelabel, 
                             ylabel, funhandle, args, params):
    #colors = ['k', 'r', 'b']
    fig, ax = plt.subplots(1,1,figsize = (5,6))
    
    x_max = params["x_max"]
    x_inputs = np.arange(1,x_max+1,1)
    for i,param in enumerate(param_vals):
        params[paramkey] = param
        out_vec = funhandle(x_inputs, *args, **params)
        ax.plot(x_inputs, out_vec, c=colors[i], label = linelabel %param)
    format_ax(ax, xlab = r'Group Size $x$', ylab = ylabel, if_legend=True)
    return fig, ax
    

# Base parameters

In [4]:
# base parameter and population size names
P_reg = 5
N1_reg = 1; N2_reg = 1
params_reg = dict(η1 = 0.2, η2 = 0.7, A1 = 0.5, β1 = 7, β2 = 2, H1=0, H2=0, 
                  α1_of_1=0.05, α2_of_1=0.95, 
                  s1=2, s2=2, α2_fun_type = 'constant',
                  x_max = 10, ξ = 2, d = 100,
                 Tx = 10, r = 0, γ = 0)



In [9]:
P = P_reg; N1 = N1_reg; N2 = N2_reg;
params = params_reg.copy()
x_max = params['x_max']
initialstate = np.array([P, *np.zeros(x_max - 1)])
t_f = 10


tsteps = 100
T=0
F_of_x_vec = initialstate
group_formation_model_non_dim(T, F_of_x_vec,P,N1,N2, params)

out = solve_ivp(group_formation_model_non_dim, [0, t_f], initialstate, method="LSODA",
                args=(P, N1, N2, params))


In [10]:
out

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.405e-05 ...  9.265e+00  1.000e+01]
        y: [[ 5.000e+00  5.000e+00 ...  3.000e-01  2.724e-01]
            [ 0.000e+00  3.162e-05 ...  4.687e-01  4.553e-01]
            ...
            [ 0.000e+00  0.000e+00 ...  3.429e-125  2.432e-125]
            [ 0.000e+00  0.000e+00 ...  2.196e-166  1.500e-166]]
      sol: None
 t_events: None
 y_events: None
     nfev: 95
     njev: 0
      nlu: 0

In [7]:
initialstate

array([5., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
    x_max = params["x_max"]
    if not isinstance(initialstate,np.ndarray): # otherwise it's an array, and intiial state is given
        if np.isnan(initialstate):
            initialstate = np.array([P, *np.zeros(x_max - 1)])

    final_f_of_x = np.zeros((len(param_vec), x_max))
    at_equilibrium = np.zeros(len(param_vec))
    dfdt_final = np.zeros((len(param_vec), x_max))
    
    # run model for each parameter value
    for i, param in enumerate(param_vec):
        # put new parameter values into params dictionary
        if isinstance(param_key, list): # in case want to set multiple params to same value
            for pkey in param_key:
                params[pkey] = param
        else:
            params[param_key] = param

        # run the model using LSODA, like in odeint?
        #tsteps = 100
        #t_eval =  np.arange(0.0, t_f, delta_t)
        #out = solve_ivp(group_formation_model_separate, [0, t_f], initialstate, method="RK45",
        #        args=(P, N1, N2, pop_process, params))
        out = solve_ivp(group_formation_model_non_dim, [0, t_f], initialstate, method="LSODA",
                args=(P, N1, N2, params))
        final_distribution = np.transpose(out.y)[-1]
        
        # check at equilibrium
        dfdt_final[i,:], at_equilibrium[i] = check_at_equilibrium(final_distribution, P, N1, N2,pop_process,**params)
        
        
        final_f_of_x[i,:] = final_distribution
        

    return final_f_of_x, at_equilibrium, dfdt_final

In [1]:
params = params_reg.copy()



out = plot_f_of_x(P=P_reg, N1=N1_reg, N2=N2_reg,
                  param_vec = [2], param_key="s1", 
            linelabel = r'$s_1=$'+'%d', params = params_reg.copy(), pop_process = False, 
                delta_t = 0.01, t_f = 5000, initialstate = np.nan)

NameError: name 'params_reg' is not defined